In [ ]:
"""
Jason Qin

Architecture and hyperparameter search

Adapted from: https://github.com/cs230-stanford/cs230-code-examples/blob/master/pytorch/vision/search_hyperparams.py
"""

In [2]:
import numpy as np
import argparse
import os
from subprocess import check_call
import sys
import re
import matplotlib.pyplot as plt

In [ ]:
"""Test different architectures"""
PYTHON = sys.executable

# encoders = ['resnet50', 'resnet101']
# decoders = ['c1', 'upernet']
encoders = ['resnet50']
decoders = ['upernet']
# train = ['', '2']
train = ['3']
trainLists = ['train_list_10x_upsampling.odgt', 'train_list_100x_upsampling.odgt']

for encoder in encoders:
    for decoder in decoders:
        for t in train:
            cmd = ('CUDA_VISIBLE_DEVICES="0,1" python /home/ky_aneur/cs230/code/resnet/semantic-segmentation-pytorch/train{train}.py --num_gpus 2 --num_class 2 --root_data '
                   '/data2/yeom/ky_aneur/sah_png/train --list_train ~/cs230/code/cta-scripts/train_list_40x_upsampling.odgt --num_epoch 20 '
                   '--id {encoder}_{decoder}_train{train} --ckpt /data2/yeom/ky_aneur/results/ckpt/loss_weights/ --arch_encoder {encoder} --arch_decoder {decoder} --optim adam --epoch_iters 500').format(train=t, encoder=encoder, decoder=decoder)
            if 'upernet' in cmd:
                cmd = cmd + ' --segm_downsampling_rate 4 --padding_constant 32'
            output = '/home/ky_aneur/cs230/code/resnet/semantic-segmentation-pytorch/training_outputs/loss_weights/{}_{}_train{train}.txt'.format(encoder, decoder, train=t)
            print(cmd)
            print(output)
            with open(output, 'w') as f:
                check_call(cmd, shell=True, stdout=f)


for encoder in encoders:
    for decoder in decoders:
        for tl in trainLists:
            t = ''
            cmd = ('CUDA_VISIBLE_DEVICES="0,1" python /home/ky_aneur/cs230/code/resnet/semantic-segmentation-pytorch/train{train}.py --num_gpus 2 --num_class 2 --root_data '
                   '/data2/yeom/ky_aneur/sah_png/train --list_train ~/cs230/code/cta-scripts/{tl} --num_epoch 20 '
                   '--id {encoder}_{decoder}_train{train} --ckpt /data2/yeom/ky_aneur/results/ckpt/upsampling/ --arch_encoder {encoder} --arch_decoder {decoder} --optim adam --epoch_iters 500').format(train=t, tl=tl, encoder=encoder, decoder=decoder)
            if 'upernet' in cmd:
                cmd = cmd + ' --segm_downsampling_rate 4 --padding_constant 32'
            output = '/home/ky_aneur/cs230/code/resnet/semantic-segmentation-pytorch/training_outputs/upsampling/{}_{}_t{tl}.txt'.format(encoder, decoder, tl = tl)
            print(cmd)
            print(output)
            with open(output, 'w') as f:
                check_call(cmd, shell=True, stdout=f)

In [47]:
"""Test different hyperparameters"""
PYTHON = sys.executable

encoders = ['resnet50']
decoders = ['upernet']
learningRates = [2e-3, 2e-1]

for encoder in encoders:
    for decoder in decoders:
        for lr in learningRates:
            cmd = ('CUDA_VISIBLE_DEVICES="0,1" python /home/ky_aneur/cs230/code/resnet/semantic-segmentation-pytorch/train3.py --num_gpus 2 --num_class 2 --segm_downsampling_rate 4 --padding_constant 32 --root_data '
                   '/data2/yeom/ky_aneur/sah_png/train --lr_decoder {lr} --list_train /home/ky_aneur/cs230/code/cta-scripts/train_list_40x_upsampling.odgt --num_epoch 20 '
                   '--id {encoder}_{decoder}_{lr} --arch_encoder {encoder} --arch_decoder {decoder} --optim adam --epoch_iters 500 --ckpt /data2/yeom/ky_aneur/results/ckpt/learning_rate/').format(lr=lr, encoder=encoder, decoder=decoder)
            output = '/home/ky_aneur/cs230/code/resnet/semantic-segmentation-pytorch/training_outputs/learning_rate/{encoder}_{decoder}_{lr}.txt'.format(encoder=encoder, decoder=decoder, lr=lr)
            print(cmd)
            print(output)
            with open(output, 'w') as f:
                check_call(cmd, shell=True, stdout=f)


        

CUDA_VISIBLE_DEVICES="0,1" python /home/ky_aneur/cs230/code/resnet/semantic-segmentation-pytorch/train3.py --num_gpus 2 --num_class 2 --segm_downsampling_rate 4 --padding_constant 32 --root_data /data2/yeom/ky_aneur/sah_png/train --lr_decoder 0.002 --list_train /home/ky_aneur/cs230/code/cta-scripts/train_list_40x_upsampling.odgt --num_epoch 20 --id resnet50_upernet_0.002 --arch_encoder resnet50 --arch_decoder upernet --optim adam --epoch_iters 500 --ckpt /data2/yeom/ky_aneur/results/ckpt/learning_rate/
/home/ky_aneur/cs230/code/resnet/semantic-segmentation-pytorch/training_outputs/learning_rate/resnet50_upernet_0.002.txt
CUDA_VISIBLE_DEVICES="0,1" python /home/ky_aneur/cs230/code/resnet/semantic-segmentation-pytorch/train3.py --num_gpus 2 --num_class 2 --segm_downsampling_rate 4 --padding_constant 32 --root_data /data2/yeom/ky_aneur/sah_png/train --lr_decoder 0.2 --list_train /home/ky_aneur/cs230/code/cta-scripts/train_list_40x_upsampling.odgt --num_epoch 20 --id resnet50_upernet_0.2 -